## Q1. What is Gradient Boosting Regression?

# Gradient Boosting Regression

Gradient Boosting Regression is a machine learning technique used for regression tasks, where the goal is to predict continuous numeric values. It is an ensemble learning method that builds a strong predictive model by sequentially adding weak learners (typically decision trees) to the ensemble.

## Workflow:

1. **Initialization**: The process starts with an initial prediction, which is often the mean (or median) of the target variable for all training samples.

2. **Building the Ensemble**:
   - A weak learner (usually a decision tree) is trained to predict the residuals (the difference between the actual target values and the current predictions).
   - The residuals are used as the new target variable for training the next weak learner.
   - Each subsequent weak learner is trained to predict the residuals of the previous model.

3. **Combining Predictions**:
   - The predictions of all weak learners are combined to make the final prediction.
   - The final prediction is the sum of the initial prediction and the predictions of all weak learners.

4. **Gradient Descent Optimization**:
   - During the training process, instead of directly fitting the residuals, Gradient Boosting uses gradient descent optimization to minimize a loss function (e.g., mean squared error).
   - The gradients of the loss function with respect to the predictions are computed, and the subsequent weak learner is trained to predict these gradients.

5. **Regularization**:
   - Gradient Boosting Regression often includes regularization techniques to prevent overfitting, such as limiting the maximum depth of the decision trees, adding a shrinkage parameter (learning rate), and using subsampling of the data.

6. **Stopping Criteria**:
   - The training process continues until a predefined number of weak learners are added to the ensemble or until a stopping criterion (e.g., no improvement on the validation set) is met.

Gradient Boosting Regression is known for its high predictive accuracy and robustness to overfitting. It can effectively capture complex nonlinear relationships in the data and is widely used in various regression tasks, such as predicting house prices, stock prices, and customer churn rates. Popular implementations of Gradient Boosting Regression include XGBoost, LightGBM, and CatBoost.


## Q2. Implement a simple gradient boosting algorithm from scratch using Python and NumPy. Use a simple regression problem as an example and train the model on a small dataset. Evaluate the model's performance using metrics such as mean squared error and R-squared.

In [14]:
import numpy as np

class DecisionTreeRegressor:
    def __init__(self, max_depth=3):
        self.max_depth = max_depth

    def fit(self, X, y):
        self.tree = self._build_tree(X, y, depth=0)
        
    def _build_tree(self, X, y, depth):
        if depth == self.max_depth or len(np.unique(y)) == 1:
            return np.mean(y)
        else:
            feature_idx, threshold, split_X_left, split_X_right, split_y_left, split_y_right = self._find_best_split(X, y)
            if feature_idx is None:
                return np.mean(y)
            else:
                tree = {}
                tree['feature_idx'] = feature_idx
                tree['threshold'] = threshold
                tree['left'] = self._build_tree(split_X_left, split_y_left, depth + 1)
                tree['right'] = self._build_tree(split_X_right, split_y_right, depth + 1)
                return tree
                
    def _find_best_split(self, X, y):
        best_mse = float('inf')
        feature_idx, threshold = None, None
        split_X_left, split_X_right = None, None  # Initialize variables here
        split_y_left, split_y_right = None, None  # Initialize variables here
        for i in range(X.shape[1]):
            for val in np.unique(X[:, i]):
                left_idx = X[:, i] <= val
                right_idx = X[:, i] > val
                if len(np.unique(y[left_idx])) < 2 or len(np.unique(y[right_idx])) < 2:
                    continue
                mse = self._mse(y[left_idx]) + self._mse(y[right_idx])
                if mse < best_mse:
                    best_mse = mse
                    feature_idx, threshold = i, val
                    split_X_left, split_X_right = X[left_idx], X[right_idx]
                    split_y_left, split_y_right = y[left_idx], y[right_idx]
        return feature_idx, threshold, split_X_left, split_X_right, split_y_left, split_y_right
                
    def _mse(self, y):
        return np.mean((y - np.mean(y))**2)
    
    def predict(self, X):
        return np.array([self._traverse_tree(x, self.tree) for x in X])
    
    def _traverse_tree(self, x, tree):
        if isinstance(tree, dict):
            if x[tree['feature_idx']] <= tree['threshold']:
                return self._traverse_tree(x, tree['left'])
            else:
                return self._traverse_tree(x, tree['right'])
        else:
            return tree

class GradientBoostingRegressor:
    def __init__(self, n_estimators=100, learning_rate=0.1, max_depth=3):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.trees = []

    def fit(self, X, y):
        residuals = y.copy()
        for _ in range(self.n_estimators):
            tree = DecisionTreeRegressor(max_depth=self.max_depth)
            tree.fit(X, residuals)
            self.trees.append(tree)
            predictions = tree.predict(X)
            residuals -= self.learning_rate * predictions

    def predict(self, X):
        predictions = np.zeros(len(X))
        for tree in self.trees:
            predictions += self.learning_rate * tree.predict(X)
        return predictions

# Example usage:
# Generate synthetic data
np.random.seed(0)
X = np.random.rand(100, 1) * 10
y = 3 * X.squeeze() + np.random.randn(100) * 2

# Train-test split
X_train, X_test = X[:80], X[80:]
y_train, y_test = y[:80], y[80:]

# Train Gradient Boosting Regressor
gb_regressor = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3)
gb_regressor.fit(X_train, y_train)

# Evaluate the model
from sklearn.metrics import mean_squared_error, r2_score
y_pred = gb_regressor.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("Mean Squared Error:", mse)
print("R-squared:", r2)


Mean Squared Error: 5.626315813710226
R-squared: 0.9179828296718999


## Q3. Experiment with different hyperparameters such as learning rate, number of trees, and tree depth to optimise the performance of the model. Use grid search or random search to find the best hyperparameters

In [21]:
from sklearn.base import BaseEstimator, RegressorMixin

class SklearnCompatibleGradientBoostingRegressor(BaseEstimator, RegressorMixin):
    def __init__(self, n_estimators=100, learning_rate=0.1, max_depth=3):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.trees = []

    def fit(self, X, y):
        residuals = y.copy()
        for _ in range(self.n_estimators):
            tree = DecisionTreeRegressor(max_depth=self.max_depth)
            tree.fit(X, residuals)
            self.trees.append(tree)
            predictions = tree.predict(X)
            residuals -= self.learning_rate * predictions

    def predict(self, X):
        predictions = np.zeros(len(X))
        for tree in self.trees:
            predictions += self.learning_rate * tree.predict(X)
        return predictions

    def get_params(self, deep=True):
        return {
            'n_estimators': self.n_estimators,
            'learning_rate': self.learning_rate,
            'max_depth': self.max_depth
        }

    def set_params(self, **params):
        for param, value in params.items():
            setattr(self, param, value)
        return self


## Q4. What is a weak learner in Gradient Boosting?

# Weak Learner in Gradient Boosting

In Gradient Boosting, a weak learner refers to a base model that is used as a component within the ensemble learning framework. Unlike in traditional machine learning methods where the emphasis is on building a single highly accurate model, in boosting algorithms like Gradient Boosting, the focus is on sequentially adding weak learners to create a strong predictive model.

A weak learner is typically a simple model that performs slightly better than random guessing on a given problem. The key characteristic of a weak learner is that it has a performance that is only slightly better than chance for binary classification tasks (e.g., accuracy slightly above 50%). For regression tasks, a weak learner may produce predictions that are only slightly correlated with the true target values.

In the context of Gradient Boosting, decision trees are commonly used as weak learners, although other types of models such as linear models or shallow neural networks can also be used. Each weak learner is trained on the residuals (the difference between the true target values and the current predictions) of the ensemble model built so far. By iteratively adding weak learners and adjusting their predictions based on the errors of the previous models, Gradient Boosting is able to gradually improve the overall performance of the ensemble.

Despite being individually weak, the combination of multiple weak learners through boosting results in a strong ensemble model that can achieve high predictive accuracy. This is because each weak learner focuses on different aspects of the data, and their collective decision-making leads to robust predictions. The sequential nature of boosting allows the algorithm to effectively adapt to complex patterns and relationships present in the data.


## Q5. What is the intuition behind the Gradient Boosting algorithm?

# Intuition Behind Gradient Boosting Algorithm

The Gradient Boosting algorithm is an ensemble learning method that combines the predictions of multiple weak learners (often decision trees) to create a strong predictive model. Here's the intuition behind how Gradient Boosting works:

1. **Ensemble Learning**: Gradient Boosting belongs to the ensemble learning family, where multiple weak learners are combined to create a more accurate and robust model than any individual learner.

2. **Sequential Training**: Unlike bagging methods like Random Forest where weak learners are trained independently in parallel, Gradient Boosting trains weak learners sequentially. Each new learner is trained to correct the errors (residuals) of the existing ensemble.

3. **Gradient Descent Optimization**: Gradient Boosting uses gradient descent optimization to minimize a loss function (e.g., mean squared error for regression tasks). At each iteration, the algorithm computes the gradients of the loss function with respect to the predictions of the current ensemble model, and updates the parameters of the weak learner in the direction that reduces the error.

4. **Gradient Boosting with Decision Trees**: Decision trees are commonly used as weak learners in Gradient Boosting. Each decision tree is trained to predict the residuals (errors) of the previous ensemble, and the predictions of all trees are combined to make the final prediction.

5. **Shrinkage (Learning Rate)**: Gradient Boosting often includes a shrinkage parameter (learning rate) to control the contribution of each weak learner to the ensemble. A smaller learning rate results in more conservative updates to the ensemble, which can improve generalization performance and help prevent overfitting.

6. **Regularization**: Gradient Boosting typically employs regularization techniques such as limiting the maximum depth of individual trees, using subsampling of the data, and adding constraints on the complexity of the weak learners to prevent overfitting.

Overall, the intuition behind Gradient Boosting is to iteratively improve the ensemble model by fitting new weak learners to the errors of the previous ensemble, using gradient descent optimization to minimize the loss function. This allows Gradient Boosting to effectively capture complex relationships in the data and achieve high predictive accuracy.


## Q6. How does Gradient Boosting algorithm build an ensemble of weak learners?

# Building an Ensemble of Weak Learners in Gradient Boosting

The Gradient Boosting algorithm builds an ensemble of weak learners sequentially. Here's how it works:

1. **Initialization**: The process starts with an initial prediction, which is often the mean (or median) of the target variable for all training samples.

2. **First Weak Learner**: The first weak learner (often a decision tree) is trained to predict the residuals (the difference between the actual target values and the initial predictions). This weak learner captures the patterns in the data that were not captured by the initial prediction.

3. **Subsequent Weak Learners**: Each subsequent weak learner is trained to predict the residuals of the current ensemble model (i.e., the difference between the actual target values and the predictions of the ensemble so far). These weak learners are trained sequentially, and each one focuses on reducing the errors made by the previous ensemble.

4. **Combining Predictions**: The predictions of all weak learners are combined to make the final prediction. Typically, the final prediction is the sum of the initial prediction and the predictions of all weak learners. By combining the predictions of multiple weak learners, Gradient Boosting produces a strong ensemble model that captures complex relationships in the data.

5. **Gradient Descent Optimization**: During the training process, Gradient Boosting uses gradient descent optimization to minimize a loss function (e.g., mean squared error). At each iteration, the algorithm computes the negative gradient of the loss function with respect to the predictions of the current ensemble model. The subsequent weak learner is then trained to predict the negative gradient, effectively moving the ensemble model in the direction that reduces the error.

6. **Regularization**: Gradient Boosting often includes regularization techniques to prevent overfitting, such as limiting the maximum depth of individual trees, using subsampling of the data, and adding constraints on the complexity of the weak learners.

By iteratively adding weak learners and adjusting their predictions based on the errors of the previous ensemble, Gradient Boosting is able to gradually improve the overall performance of the ensemble and produce highly accurate predictions.


## Q7. What are the steps involved in constructing the mathematical intuition of Gradient Boosting algorithm?

# Steps Involved in Constructing the Mathematical Intuition of Gradient Boosting Algorithm

1. **Initialize with a Constant**: Start with an initial prediction, often the mean (or median) of the target variable for all training samples.

2. **Compute Pseudo-Residuals**: Calculate the pseudo-residuals, which are the negative gradients of the loss function with respect to the current predictions.

3. **Fit a Weak Learner**: Train a weak learner (e.g., decision tree) to predict the pseudo-residuals. This weak learner captures the patterns in the data that were not captured by the initial prediction.

4. **Update Predictions**: Update the predictions by adding a fraction (learning rate) of the predictions of the weak learner to the current predictions. This step gradually improves the overall predictions of the ensemble.

5. **Repeat Steps 2-4**: Iterate the process by computing new pseudo-residuals based on the updated predictions and fitting a new weak learner to predict these pseudo-residuals. Each weak learner focuses on correcting the errors made by the ensemble so far.

6. **Combine Predictions**: Combine the predictions of all weak learners to make the final prediction. Typically, the final prediction is the sum of the initial prediction and the predictions of all weak learners.

By following these steps, Gradient Boosting constructs an ensemble of weak learners that work together to make highly accurate predictions.
